In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/HME/Temporal-Transformer

/content/drive/MyDrive/HME/Temporal-Transformer


In [ ]:
from itertools import zip_longest
import numpy as np
import random
import torch

In [ ]:
def get_frame_num(frame_name):
    l = frame_name.split('_')

    m = l[1].split('.')
    n = m[0]
    frame_num = int(n)
    return frame_num

In [ ]:
# print(get_frame_num('frame_201.jpg'))

In [ ]:
def get_j_neighbor(dataset, central_idx, one_side, j, S, video, central_frame_num):
    # subject = []
    if dataset[central_idx + j][0][0] == S:
        if dataset[central_idx - 1][0][1] == video:

            # extract the frame number
            frame_name = dataset[central_idx + j][0][2]
            frame_num = get_frame_num(frame_name)
            if frame_num == central_frame_num + 20*j:

                ##########################
                '''change here to revise the input data'''
                # make pose for an example
                subject = dataset[central_idx + j][3]
            else:
                subject = dataset[central_idx][3]
                print('the ', j,  'of frame', central_frame_num, 'not found')
        
        else:
            subject = dataset[central_idx][3]
            print('the ', j,  'of frame ', central_frame_num, 'not found')
    else:
        subject = dataset[central_idx][3]
        print('the ', j,  'of frame ', central_frame_num, 'not found')
    return subject

In [ ]:
def get_j_neighbor_without_print(dataset, central_idx, one_side, j, S, video, central_frame_num):
    # subject = []
    if dataset[central_idx + j][0][0] == S:
        if dataset[central_idx - 1][0][1] == video:

            # extract the frame number
            frame_name = dataset[central_idx + j][0][2]
            frame_num = get_frame_num(frame_name)
            if frame_num == central_frame_num + 20*j:

                ##########################
                '''change here to revise the input data'''
                # make pose for an example
                subject = dataset[central_idx + j][3]
            else:
                subject = dataset[central_idx][3]
                # print('the ', j,  'of frame', central_frame_num, 'not found')
        
        else:
            subject = dataset[central_idx][3]
            # print('the ', j,  'of frame ', central_frame_num, 'not found')
    else:
        subject = dataset[central_idx][3]
        # print('the ', j,  'of frame ', central_frame_num, 'not found')
    return subject

In [ ]:
def get_j_neighbor_with_index(dataset, central_idx, one_side, j, S, video, central_frame_num, type=3):
    # subject = []
    if dataset[central_idx + j][0][0] == S:
        if dataset[central_idx - 1][0][1] == video:

            # extract the frame number
            frame_name = dataset[central_idx + j][0][2]
            frame_num = get_frame_num(frame_name)
            if frame_num == central_frame_num + 20*j:

                ##########################
                '''change here to revise the input data'''
                # make pose for an example
                subject = dataset[central_idx + j][type]
            else:
                subject = dataset[central_idx][type]
                # print('the ', j,  'of frame', central_frame_num, 'not found')
        
        else:
            subject = dataset[central_idx][type]
            # print('the ', j,  'of frame ', central_frame_num, 'not found')
    else:
        subject = dataset[central_idx][type]
        # print('the ', j,  'of frame ', central_frame_num, 'not found')
    return subject

In [ ]:
class Dataloader_zzx:
    def __init__(self, dataset, batch_size=512, receptive_field=9):
        # dataset = np.transpose(dataset, (1,0))    # not needed for the newly formed trainingdata
        self.tol_num_data = len(dataset)
        self.batch_size = batch_size
        self.n_batches = self.tol_num_data // self.batch_size
        self.receptive_field = receptive_field
        self.dataset = dataset
        self.one_side = receptive_field // 2      # how many frames to search in one side
    
    def generator(self):
        print("dividing data......")
        for i in range(self.n_batches):
            # local batch and labels
            indices = random.sample(range(0,self.tol_num_data), self.batch_size)
            central_images = self.dataset[indices]      # retrieve data by raws
            one_side = self.one_side

            current_batch = []
            subject = []
            # generate the data from an single frame and combine them together.
            for i, img in enumerate(central_images):

                S, video, frame = img[0]        # Ex: S = 'S1', video = 'Posing 1.54138969', frame = 'frame_201.jpg'
                central_frame_num = get_frame_num(frame)
                # get the location of the central frame in the dataset.
                central_idx = indices[i]

                # repeat the central frame if can't find the neighbor frames.
                if central_idx <= self.one_side or central_idx >= self.tol_num_data - self.one_side:
                    '''change here'''
                    subject_central = img[3]
                    subject = [subject_central] * self.one_side * 2   # repeat the element for 2*one_side times.
                    # print('******************', len(subject))
                    subject_np = subject[0]
                    # print(len(subject), '\n', central_frame_num)
                    for frame_np in subject[1:]:
                        subject_np = np.vstack([subject_np, frame_np])
                    
                    current_batch.append(subject_np)

                    subject = [] #set subject clear
                    
                    # print('get neighbors for', central_frame_num,'\n', '####################')

                    continue
                    # break
                
                # generate the sequence list: the left and right of the central frame
                else:
                    for j in [x for x in range(-self.one_side, 0)] + [x+1 for x in range(0, self.one_side)]:    # without the central frame
                        subject.append(get_j_neighbor_with_index(self.dataset, central_idx, self.one_side, j, S, video, central_frame_num))

                

                # convert the list (len(6)) to array that is (6*)   (85)*9 -> (9,85)
                # stack list elements as row in numpy.array
                subject_np = subject[0]
                # print(len(subject), '\n', central_frame_num)
                for frame_np in subject[1:]:
                    subject_np = np.vstack([subject_np, frame_np])
                
                current_batch.append(subject_np)

                subject = [] #set subject clear
                



            ######## pose_and_shape ##########
            # convert the list to numpy (9,85) -> (512,9,85)
            batch_np = current_batch[0]
            batch_np = np.expand_dims(batch_np, axis=0)
            for bundle in current_batch[1:]:
                bundle = np.expand_dims(bundle, axis=0)
                batch_np = np.vstack([batch_np, bundle])





            # numpy -> tensor
            training_set_batch = torch.from_numpy(batch_np)
            central_gt_batch = torch.from_numpy(central_gt_np)
            yield training_set_batch      # yield the result


In [ ]:
class Dataloader_zzx_video:
    def __init__(self, video_path, receptive_field=9):
        # dataset = np.transpose(dataset, (1,0))    # not needed for the newly formed trainingdata
        
        # data =[]
        # for filename in os.listdir(dataset_folder):
        #     file_path = os.path.join(folder_name, filename)
        #     data.append(np.load(file_path, allow_pickle=True).transpose(1,0))
        
        # dataset = data[0]
        # for i in data[1:]:
        #     dataset = np.vstack([dataset, i])   
        dataset = np.load(video_path, allow_pickle=True).transpose(1,0)
        print(len(dataset))

        self.tol_num_data = len(dataset)
        self.receptive_field = receptive_field
        self.dataset = dataset
        self.one_side = receptive_field // 2      # how many frames to search in one side
    
    def generator(self):
        print("dividing data......")
        # local batch and labels
        indices = [x for x in range(self.tol_num_data)]
        # central_images = self.dataset[indices]      # retrieve data by raws
        one_side = self.one_side



        current_batch = []
        subject = []
        central_gt = []
        central_name = []
        # generate the data from an single frame and combine them together.
        for i, img in enumerate(self.dataset):

            # import pdb
            # pdb.set_trace()
            central_gt.append(np.expand_dims(img[3], axis=0))
            central_name.append(np.expand_dims(img[0], axis=0))


            S, video, frame = img[0]        # Ex: S = 'S1', video = 'Posing 1.54138969', frame = 'frame_201.jpg'
            central_frame_num = get_frame_num(frame)
            # get the location of the central frame in the dataset.
            central_idx = indices[i]

            # repeat the central frame if can't find the neighbor frames.
            if central_idx <= self.one_side or central_idx >= self.tol_num_data - self.one_side:
                '''change here'''
                subject_central = img[3]
                subject = [subject_central] * self.one_side * 2   # repeat the element for 2*one_side times.
                
                subject_np = subject[0]
                for frame_np in subject[1:]:
                    subject_np = np.vstack([subject_np, frame_np])
                
                current_batch.append(subject_np)

                subject = [] #set subject clear
                
                

                continue
              
            
            # generate the sequence list: the left and right of the central frame
            else:
                for j in [x for x in range(-self.one_side, 0)] + [x+1 for x in range(0, self.one_side)]:    # without the central frame
                    subject.append(get_j_neighbor_without_print(self.dataset, central_idx, self.one_side, j, S, video, central_frame_num))
        
            subject_np = subject[0]

            for frame_np in subject[1:]:
                subject_np = np.vstack([subject_np, frame_np])
            
            current_batch.append(subject_np)

            subject = [] #set subject clear
                



        ######## pose_and_shape ##########
        # convert the list to numpy (9,85) -> (512,9,85)
        batch_np = current_batch[0]
        batch_np = np.expand_dims(batch_np, axis=0)
        for bundle in current_batch[1:]:
            bundle = np.expand_dims(bundle, axis=0)
            batch_np = np.vstack([batch_np, bundle])
        # numpy -> tensor
        training_set_batch = torch.from_numpy(batch_np)

        ######## central GT ###########
        central_gt_np = central_gt[0]
        central_gt_np = np.expand_dims(central_gt_np, axis = 0)
        for central in central_gt[1:]:
            central = np.expand_dims(central, axis=0)
            central_gt_np = np.vstack([central_gt_np, central])
        # numpy -> tensor
        central_gt_batch = torch.from_numpy(central_gt_np)



        ####### central name #########
        yield training_set_batch, central_gt_batch, central_name      # yield the result


In [ ]:
### specific video experiment
import os

hm_path = '/content/drive/MyDrive/HME/H36M Data/Cropped Images/HMR Outputs/Structure 2_gt'

subject = 'S6'
video_name = 'Discussion 1.54138969' + '_resultsgt.npy'
# Directions 1.58860488_resultsgt.npy
# Discussion 1.54138969, Greeting 1.58860488, \Sitting 1.58860488

subject_folder = os.path.join(hm_path, subject)
video_path = os.path.join(subject_folder, video_name)

dataloader = Dataloader_zzx_video(video_path, receptive_field=9)

for i, central_i, central_name in dataloader.generator():
    print(central_name
)

NameError: ignored

In [ ]:
a = np.ones((10,1))
b = np.linspace(0,5,10)
indices = random.sample(range(0,10), 3)
print(indices)

c = [x for x in range(10)]
print(c)

[5, 0, 7]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
class Dataloader_zzx_central_gt_frame:
    def __init__(self, dataset, batch_size=512, receptive_field=9):
        # dataset = np.transpose(dataset, (1,0))    # not needed for the newly formed trainingdata
        self.tol_num_data = len(dataset)
        self.batch_size = batch_size
        self.n_batches = self.tol_num_data // self.batch_size
        self.receptive_field = receptive_field
        self.dataset = dataset
        self.one_side = receptive_field // 2      # how many frames to search in one side
    
    def generator(self):
        print("dividing data......")
        for i in range(self.n_batches):
            # local batch and labels
            indices = random.sample(range(0,self.tol_num_data), self.batch_size)
            central_images = self.dataset[indices]      # retrieve data by raws
            one_side = self.one_side

            current_batch = []
            subject = []
            central_gt = []
            central_name = []
            # generate the data from an single frame and combine them together.
            for i, img in enumerate(central_images):

                # import pdb
                # pdb.set_trace()
                central_gt.append(np.expand_dims(img[3], axis=0))
                central_name.append(np.expand_dims(img[0], axis=0))


                S, video, frame = img[0]        # Ex: S = 'S1', video = 'Posing 1.54138969', frame = 'frame_201.jpg'
                central_frame_num = get_frame_num(frame)
                # get the location of the central frame in the dataset.
                central_idx = indices[i]

                # repeat the central frame if can't find the neighbor frames.
                if central_idx <= self.one_side or central_idx >= self.tol_num_data - self.one_side:
                    '''change here'''
                    subject_central = img[3]
                    subject = [subject_central] * self.one_side * 2   # repeat the element for 2*one_side times.
                    
                    subject_np = subject[0]
                    for frame_np in subject[1:]:
                        subject_np = np.vstack([subject_np, frame_np])
                    
                    current_batch.append(subject_np)

                    subject = [] #set subject clear
                    
                   

                    continue
                  
                
                # generate the sequence list: the left and right of the central frame
                else:
                    for j in [x for x in range(-self.one_side, 0)] + [x+1 for x in range(0, self.one_side)]:    # without the central frame
                        subject.append(get_j_neighbor_without_print(self.dataset, central_idx, self.one_side, j, S, video, central_frame_num))
           
                subject_np = subject[0]

                for frame_np in subject[1:]:
                    subject_np = np.vstack([subject_np, frame_np])
                
                current_batch.append(subject_np)

                subject = [] #set subject clear
                



            ######## pose_and_shape ##########
            # convert the list to numpy (9,85) -> (512,9,85)
            batch_np = current_batch[0]
            batch_np = np.expand_dims(batch_np, axis=0)
            for bundle in current_batch[1:]:
                bundle = np.expand_dims(bundle, axis=0)
                batch_np = np.vstack([batch_np, bundle])
            # numpy -> tensor
            training_set_batch = torch.from_numpy(batch_np)

            ######## central GT ###########
            central_gt_np = central_gt[0]
            central_gt_np = np.expand_dims(central_gt_np, axis = 0)
            for central in central_gt[1:]:
                central = np.expand_dims(central, axis=0)
                central_gt_np = np.vstack([central_gt_np, central])
            # numpy -> tensor
            central_gt_batch = torch.from_numpy(central_gt_np)



            ####### central name #########
            yield training_set_batch, central_gt_batch, central_name      # yield the result


In [ ]:
class Dataloader_zzx_central_gt:
    def __init__(self, dataset, batch_size=512, receptive_field=9):
        # dataset = np.transpose(dataset, (1,0))    # not needed for the newly formed trainingdata
        self.tol_num_data = len(dataset)
        self.batch_size = batch_size
        self.n_batches = self.tol_num_data // self.batch_size
        self.receptive_field = receptive_field
        self.dataset = dataset
        self.one_side = receptive_field // 2      # how many frames to search in one side
    
    def generator(self):
        print("dividing data......")
        for i in range(self.n_batches):
            # local batch and labels
            indices = random.sample(range(0,self.tol_num_data), self.batch_size)
            central_images = self.dataset[indices]      # retrieve data by raws
            one_side = self.one_side

            current_batch = []
            subject = []
            central_gt = []
            # generate the data from an single frame and combine them together.
            for i, img in enumerate(central_images):

                # import pdb
                # pdb.set_trace()
                central_gt.append(np.expand_dims(img[3], axis=0))



                S, video, frame = img[0]        # Ex: S = 'S1', video = 'Posing 1.54138969', frame = 'frame_201.jpg'
                central_frame_num = get_frame_num(frame)
                # get the location of the central frame in the dataset.
                central_idx = indices[i]

                # repeat the central frame if can't find the neighbor frames.
                if central_idx <= self.one_side or central_idx >= self.tol_num_data - self.one_side:
                    '''change here'''
                    subject_central = img[3]
                    subject = [subject_central] * self.one_side * 2   # repeat the element for 2*one_side times.
                    
                    subject_np = subject[0]
                    for frame_np in subject[1:]:
                        subject_np = np.vstack([subject_np, frame_np])
                    
                    current_batch.append(subject_np)

                    subject = [] #set subject clear
                    
                   

                    continue
                  
                
                # generate the sequence list: the left and right of the central frame
                else:
                    for j in [x for x in range(-self.one_side, 0)] + [x+1 for x in range(0, self.one_side)]:    # without the central frame
                        subject.append(get_j_neighbor_without_print(self.dataset, central_idx, self.one_side, j, S, video, central_frame_num))
           
                subject_np = subject[0]

                for frame_np in subject[1:]:
                    subject_np = np.vstack([subject_np, frame_np])
                
                current_batch.append(subject_np)

                subject = [] #set subject clear
                



            ######## pose_and_shape ##########
            # convert the list to numpy (9,85) -> (512,9,85)
            batch_np = current_batch[0]
            batch_np = np.expand_dims(batch_np, axis=0)
            for bundle in current_batch[1:]:
                bundle = np.expand_dims(bundle, axis=0)
                batch_np = np.vstack([batch_np, bundle])
            # numpy -> tensor
            training_set_batch = torch.from_numpy(batch_np)

            ######## central GT ###########
            central_gt_np = central_gt[0]
            central_gt_np = np.expand_dims(central_gt_np, axis = 0)
            for central in central_gt[1:]:
                central = np.expand_dims(central, axis=0)
                central_gt_np = np.vstack([central_gt_np, central])
            # numpy -> tensor
            central_gt_batch = torch.from_numpy(central_gt_np)
            yield training_set_batch, central_gt_batch      # yield the result


In [ ]:
class Dataloader_zzx_central_hm36_gt_tag:
    def __init__(self, dataset, batch_size=512, receptive_field=9):
        # dataset = np.transpose(dataset, (1,0))    # not needed for the newly formed trainingdata
        self.tol_num_data = len(dataset)
        self.batch_size = batch_size
        self.n_batches = self.tol_num_data // self.batch_size
        self.receptive_field = receptive_field
        self.dataset = dataset
        self.one_side = receptive_field // 2      # how many frames to search in one side
    
    def generator(self):
        print("dividing data......")
        for i in range(self.n_batches):
            # local batch and labels
            indices = random.sample(range(0,self.tol_num_data), self.batch_size)
            central_images = self.dataset[indices]      # retrieve data by raws
            one_side = self.one_side

            current_batch = []
            subject = []
            central_gt = []
            central_3d_gt = []
            central_cams = []
            central_name = []
            # generate the data from an single frame and combine them together.
            for i, img in enumerate(central_images):

                # import pdb
                # pdb.set_trace()
                central_gt.append(np.expand_dims(img[3], axis=0))
                central_3d_gt.append(np.expand_dims(img[6], axis = 0))
                central_cams.append(np.expand_dims(img[4],axis=0))
                central_name.append(np.expand_dims(img[0], axis=0))

                S, video, frame = img[0]        # Ex: S = 'S1', video = 'Posing 1.54138969', frame = 'frame_201.jpg'
                central_frame_num = get_frame_num(frame)
                # get the location of the central frame in the dataset.
                central_idx = indices[i]

                # repeat the central frame if can't find the neighbor frames.
                if central_idx <= self.one_side or central_idx >= self.tol_num_data - self.one_side:
                    '''change here'''
                    subject_central = img[3]
                    subject = [subject_central] * self.one_side * 2   # repeat the element for 2*one_side times.
                    
                    subject_np = subject[0]
                    for frame_np in subject[1:]:
                        subject_np = np.vstack([subject_np, frame_np])
                    
                    current_batch.append(subject_np)

                    subject = [] #set subject clear
                    
                   

                    continue
                  
                
                # generate the sequence list: the left and right of the central frame
                else:
                    for j in [x for x in range(-self.one_side, 0)] + [x+1 for x in range(0, self.one_side)]:    # without the central frame
                        subject.append(get_j_neighbor_without_print(self.dataset, central_idx, self.one_side, j, S, video, central_frame_num))
           
                subject_np = subject[0]

                for frame_np in subject[1:]:
                    subject_np = np.vstack([subject_np, frame_np])
                
                current_batch.append(subject_np)

                subject = [] #set subject clear
                



            ######## pose_and_shape ##########
            # convert the list to numpy (9,85) -> (512,9,85)
            batch_np = current_batch[0]
            batch_np = np.expand_dims(batch_np, axis=0)
            for bundle in current_batch[1:]:
                bundle = np.expand_dims(bundle, axis=0)
                batch_np = np.vstack([batch_np, bundle])
            # numpy -> tensor
            training_set_batch = torch.from_numpy(batch_np)

            ######## central GT ###########
            central_gt_np = central_gt[0]
            central_gt_np = np.expand_dims(central_gt_np, axis = 0)
            for central in central_gt[1:]:
                central = np.expand_dims(central, axis=0)
                central_gt_np = np.vstack([central_gt_np, central])
            # numpy -> tensor
            central_gt_batch = torch.from_numpy(central_gt_np)

            ####### central 3d GT#########
            central_3d_gt_np = central_3d_gt[0]
            central_3d_gt_np = np.expand_dims(central_3d_gt_np, axis = 0)  
            for central_3d in central_3d_gt[1:]:
                central_3d = np.expand_dims(central_3d, axis=0)
                central_3d_gt_np = np.vstack([central_3d_gt_np, central_3d])
            central_3d_gt_batch = torch.from_numpy(central_3d_gt_np)


            ####### cams ###########
            cams_np = central_cams[0]
            cams_np = np.expand_dims(cams_np, axis = 0)
            for cam in central_cams[1:]:
                cam = np.expand_dims(cam, axis=0)
                cams_np = np.vstack([cams_np, cam])
            cams_batch = torch.from_numpy(cams_np)

            yield training_set_batch, central_gt_batch, central_3d_gt_batch, cams_batch, central_name      # yield the result


In [ ]:
import numpy as np
data = np.load('trainingdata.npy', allow_pickle=True)
dataloader = Dataloader_zzx_central_hm36_gt_tag(data, batch_size=512)
for i, central, central_3d, cams, names in dataloader.generator():
    print(cams.shape)     # expected
    break

# central.squeeze().shape

FileNotFoundError: ignored

In [ ]:
import numpy as np
data = np.load('trainingdata.npy', allow_pickle=True)
dataloader = Dataloader_zzx_central_gt_frame(data, batch_size=512)
for i, central, name in dataloader.generator():
    print(name)     # expected
    break

# central.squeeze().shape

dividing data......
[array([['S11', 'Posing.54138969', 'frame_721.jpg']], dtype='<U15'), array([['S11', 'Posing.54138969', 'frame_561.jpg']], dtype='<U15'), array([['S9', 'Waiting 1.55011271', 'frame_221.jpg']], dtype='<U18'), array([['S7', 'WalkTogether 1.55011271', 'frame_1541.jpg']], dtype='<U23'), array([['S7', 'SittingDown 1.54138969', 'frame_2621.jpg']], dtype='<U22'), array([['S9', 'SittingDown.60457274', 'frame_1741.jpg']], dtype='<U20'), array([['S9', 'Greeting 1.54138969', 'frame_1601.jpg']], dtype='<U19'), array([['S5', 'WalkDog.54138969', 'frame_1961.jpg']], dtype='<U16'), array([['S1', 'WalkingDog 1.58860488', 'frame_61.jpg']], dtype='<U21'), array([['S9', 'WalkDog.60457274', 'frame_201.jpg']], dtype='<U16'), array([['S11', 'Smoking 2.54138969', 'frame_1561.jpg']], dtype='<U18'), array([['S5', 'Eating 1.54138969', 'frame_221.jpg']], dtype='<U17'), array([['S1', 'Greeting.60457274', 'frame_41.jpg']], dtype='<U17'), array([['S1', 'WalkingDog.58860488', 'frame_741.jpg']], dty

In [ ]:
# data = np.load('Posing_1.54138969_results.npy', allow_pickle=True)
# len(data)
# data[4][9]

array([ 0.07900667,  0.3255531 ,  0.7409295 ,  0.13318908,  0.37806144,
        0.01350871,  0.3620047 ,  0.21933106,  0.22547124, -0.04112568],
      dtype=float32)

In [ ]:
# ## experiment with yield
# def g(i):
#   for j in range(i):
#     yield j

# for i in g(10):
#   print(i)

# # it works

0
1
2
3
4
5
6
7
8
9


In [ ]:
# # add a row in numpy array
# import numpy
# A = numpy.array([[0, 1, 2], [0, 2, 0]])
# A = numpy.expand_dims(A,axis=0)
# print(A.shape)
# newrow = numpy.array([[0,1,2], [0,1,2]])
# newrow = numpy.expand_dims(newrow, axis=0)
# A = numpy.vstack([A, newrow])
# print(A.shape)
# print(A)

# l = [1,2,3]
# for i in l[1:]:
#   print(i)

# import torch
# A = torch.from_numpy(A)
# print(A)


# B = numpy.array([0,1,2])

# print(B.shape)
# C = numpy.array([1,2,3])
# B = numpy.vstack([B,C])
# print(B)

(1, 2, 3)
(2, 2, 3)
[[[0 1 2]
  [0 2 0]]

 [[0 1 2]
  [0 1 2]]]
2
3
tensor([[[0, 1, 2],
         [0, 2, 0]],

        [[0, 1, 2],
         [0, 1, 2]]])
(3,)
[[0 1 2]
 [1 2 3]]


In [ ]:
                ''' -1 for example '''
                # if dataset[central_idx - 1][0][0] == S:
                #     if dataset[central_idx - 1][0][1] == video:

                #         # extract the frame number
                #         frame_name = dataset[central_idx - 1][0][2]
                #         frame_num = get_frame_num(fram_name)
                #         if frame_num == central_frame_num - 20:
                #             subject.append(dataset[central_idx - 1])
                #         else:
                #             subject.append(dataset[central_idx])
                #             print('the -1 of frame', central_frame_num, 'not found')
                    
                #     else:
                #         subject.append(dataset[central_idx])
                #         print('the -1 of frame', central_frame_num, 'not found'
                # else:
                #     subject.append(dataset[central_idx])
                #     print('the -1 of frame', central_frame_num, 'not found'

In [ ]:
# # reshape the training data
# import os
# import numpy as np


# i = 0

# dir = '/content/drive/MyDrive/HME/H36M Data/Cropped Images/HMR Outputs/Structure 2_gt'
# foldernames_train = ['S1','S5','S7','S9','S11']
# for foldername in foldernames_train:
#     dir_path = dir + '/' + foldername
#     for filename in os.listdir(dir_path):
#         path = dir_path + '/' + filename
#         # print(path)
#         i += 1
#         dataset = np.load(path, allow_pickle=True)
#         dataset = np.transpose(dataset, (1,0))
#         if i == 1:  # the first one can't be stack
#             newdata = dataset
#         else:
#             newdata = np.vstack([newdata, dataset])
#         print(len(newdata))
#   print(filename, 'is the last file')
#   print(i)


    
# # print(len(dataset[:,4]))
# # np.concatenate((dataset[:,3], dataset[:,4]), axis = 1)
# # np.concatenate(dataset[0,3], dataset[0,4], axis = 0)
# # newdata = np.concatenate(dataset[:,3], dataset[:,4])

In [ ]:
# # check frame consistency
# # for i in range(len(newdata)):
# #     print(newdata[i][0])


# # save the data
# np.save('trainingdata',newdata)

In [ ]:
# dataset = np.load('Directions 1.58860488_resultsgt.npy', allow_pickle=True)
# dataset = np.transpose(dataset, (1,0))
# dataset[0][0]

['S1', 'Directions 1.58860488', 'frame_1.jpg']

In [ ]:
# dataset = np.load('trainingdata.npy', allow_pickle=True)
# dataset[0][0]

['S1', 'WalkingDog 1.54138969', 'frame_1.jpg']